In [ ]:
import keras
import tensorflow as tf
import keras.backend as K
import numpy

from keras.models import Model
from keras.layers import Dense, Input, Dropout
from sklearn.model_selection import train_test_split, ParameterGrid

# Loading data
The data is in a csv file. The first 8 columns are the features and the final binary column is the label.

In [ ]:
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

In [ ]:
X, Y = dataset[:, 0:8], dataset[:, 8]

We split the data to train and test, even though there is no testing step in this notebook. 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1)

---------
We define a function to parse the hyper-parameters so each train is in a different folder.

In [ ]:
def parse_model(batch,units,act,opt,drop):
    p = lambda t: ",".join(map(str,t))
    return f"batch={batch}_hidden={p(units)}_activation={act}_optimizer={opt}_dropout={drop:.0%}"

A functiont o create a mobdel that takes different hyper-parameters we want to test. 

In [ ]:
def create_model(batch=10, units=(5,3,3), act='relu', opt='adam', drop=0.5, i=0):
    input_0 = Input(shape=(8,))
    input_1 = Dropout(drop)(input_0)
    
    layer_0 = Dropout(drop)(Dense(units[0], input_dim=8, activation=act)(input_1))
    
    if len(units) > 1:
        for i,l in enumerate(units,1):
            exec(f"layer_{i} = Dropout({drop})(Dense({l}, activation='{act}')(layer_{i-1}))")

    final = eval(f"Dense(1, activation='sigmoid')(layer_{i})")
    
    model = Model(inputs=[input_0], outputs=[final])
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    stopping = keras.callbacks.EarlyStopping(patience=5)
    tensor = keras.callbacks.TensorBoard(log_dir='./Graph/' + parse_model(batch,units,act,opt,drop))
    
    return model,[tensor,stopping]

This function creates a model given parameters and trains it for 100 epochs or less if the early stopping kicks in.

In [ ]:
def train_model(args):
    K.clear_session()
    print("Training model:" + parse_model(**args))
    model, calls = create_model(**args)
    model.fit(X_train, Y_train, epochs=100, batch_size=args['batch'], validation_split=0.2, verbose=0, callbacks=calls)

We use the SKlearn `ParameterGrid` to generate a list of dictionaries with all the different combinations to train. 

In [ ]:
params = ParameterGrid({'batch': [10,40], 
                        'units':[(4,), (8,4)],
                        'act': ['relu', 'tanh'], 
                        'opt': ['adam','adadelta'],
                        'drop': [0.5, 0.9]})

Also, we make sure there is a folder called `Graph` otherwise we'll get an error. 

In [ ]:
!mkdir -p Graph

-------
The main loop; for each paramter configuration, we train a model. Each model will have a different folder inside `./Graph/` so tensorboard knows to treat them as different models. 

In [ ]:
for p in params:
    train_model(p)